# Building Applications with LLMs

- [Skool Link](https://www.skool.com/data-alchemy/classroom/a455582e?md=cffccd9c21af41e0bc3669610ce3bc39)
- [YouTube](https://www.youtube.com/watch?time_continue=1783&v=NYSWn1ipbgg&embeds_referring_euri=https%3A%2F%2Fwww.skool.com%2F&embeds_referring_origin=https%3A%2F%2Fwww.skool.com&source_ve_path=Mjg2NjY&feature=emb_logo)
- [LangChain Experiments](https://github.com/daveebbelaar/langchain-experiments)

In [1]:
import os
import dotenv

dotenv_path = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_path)

True

In [2]:
# --------------------------------------------------------------
# Import Modules
# --------------------------------------------------------------

import os
import json
import openai
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage

In [5]:
# --------------------------------------------------------------
# Ask ChatGPT a Question
# --------------------------------------------------------------

completion = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {
            "role": "user",
            "content": "When's the next flight from Amsterdam to New York?",
        },
    ],
)

output = completion.choices[0].message.content
print(output)

I'm sorry, but as an AI, I do not have real-time information on flight schedules. It's best to check with airlines or online flight booking platforms to get the most accurate and up-to-date information about flight schedules.


In [6]:
# --------------------------------------------------------------
# Use OpenAI’s Function Calling Feature
# --------------------------------------------------------------

function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    }
]

user_prompt = "When's the next flight from Amsterdam to New York?"

In [7]:
completion = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": user_prompt}],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
)

In [8]:
# It automatically fills the arguments with correct info based on the prompt
# Note: the function does not exist yet

output = completion.choices[0].message
print(output)


ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', name='get_flight_info'), tool_calls=None)


In [9]:
# --------------------------------------------------------------
# Add a Function
# --------------------------------------------------------------


def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    # Example output returned from an API or database
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "KLM",
        "flight": "KL643",
    }

    return json.dumps(flight_info)


In [10]:
# Use the LLM output to manually call the function
# The json.loads function converts the string to a Python object

origin = json.loads(output.function_call.arguments).get("loc_origin")
destination = json.loads(output.function_call.arguments).get("loc_destination")
params = json.loads(output.function_call.arguments)
type(params)

print(origin)
print(destination)
print(params)

# Call the function with arguments

chosen_function = eval(output.function_call.name)
flight = chosen_function(**params)

print(flight)

AMS
JFK
{'loc_origin': 'AMS', 'loc_destination': 'JFK'}
{"loc_origin": "AMS", "loc_destination": "JFK", "datetime": "2024-03-27 13:00:48.891729", "airline": "KLM", "flight": "KL643"}


In [11]:
# --------------------------------------------------------------
# Add function result to the prompt for a final answer
# --------------------------------------------------------------

# The key is to add the function output back to the messages with role: function
second_completion = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "function", "name": output.function_call.name, "content": flight},
    ],
    functions=function_descriptions,
)
response = second_completion.choices[0].message.content
print(response)

The next flight from Amsterdam (AMS) to New York (JFK) is on March 27, 2024, at 13:00. The flight is operated by KLM (flight KL643).


In [12]:
# --------------------------------------------------------------
# Include Multiple Functions
# --------------------------------------------------------------

# Expand on function descriptions (3 functions)

function_descriptions_multiple = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    },
    {
        "name": "book_flight",
        "description": "Book a flight based on flight information",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
                "datetime": {
                    "type": "string",
                    "description": "The date and time of the flight, e.g. 2023-01-01 01:01",
                },
                "airline": {
                    "type": "string",
                    "description": "The service airline, e.g. Lufthansa",
                },
            },
            "required": ["loc_origin", "loc_destination", "datetime", "airline"],
        },
    },
    {
        "name": "file_complaint",
        "description": "File a complaint as a customer",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the user, e.g. John Doe",
                },
                "email": {
                    "type": "string",
                    "description": "The email address of the user, e.g. john@doe.com",
                },
                "text": {
                    "type": "string",
                    "description": "Description of issue",
                },
            },
            "required": ["name", "email", "text"],
        },
    },
]

print(function_descriptions_multiple)

[{'name': 'get_flight_info', 'description': 'Get flight information between two locations', 'parameters': {'type': 'object', 'properties': {'loc_origin': {'type': 'string', 'description': 'The departure airport, e.g. DUS'}, 'loc_destination': {'type': 'string', 'description': 'The destination airport, e.g. HAM'}}, 'required': ['loc_origin', 'loc_destination']}}, {'name': 'book_flight', 'description': 'Book a flight based on flight information', 'parameters': {'type': 'object', 'properties': {'loc_origin': {'type': 'string', 'description': 'The departure airport, e.g. DUS'}, 'loc_destination': {'type': 'string', 'description': 'The destination airport, e.g. HAM'}, 'datetime': {'type': 'string', 'description': 'The date and time of the flight, e.g. 2023-01-01 01:01'}, 'airline': {'type': 'string', 'description': 'The service airline, e.g. Lufthansa'}}, 'required': ['loc_origin', 'loc_destination', 'datetime', 'airline']}}, {'name': 'file_complaint', 'description': 'File a complaint as a 

In [17]:
def ask_and_reply(prompt):
    """Give LLM a given prompt and get an answer."""

    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": prompt}],
        # add function calling
        functions=function_descriptions_multiple,
        function_call="auto",  # specify the function call
    )

    output = completion.choices[0].message
    return output

In [18]:
# Scenario 1: Check flight details

user_prompt = "When's the next flight from Amsterdam to New York?"
print(ask_and_reply(user_prompt))

# Get info for the next prompt

origin = json.loads(output.function_call.arguments).get("loc_origin")
destination = json.loads(output.function_call.arguments).get("loc_destination")
chosen_function = eval(output.function_call.name)
flight = chosen_function(origin, destination)

print(origin)
print(destination)
print(flight)

flight_datetime = json.loads(flight).get("datetime")
flight_airline = json.loads(flight).get("airline")

print(flight_datetime)
print(flight_airline)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', name='get_flight_info'), tool_calls=None)
AMS
JFK
{"loc_origin": "AMS", "loc_destination": "JFK", "datetime": "2024-03-27 13:10:16.057369", "airline": "KLM", "flight": "KL643"}
2024-03-27 13:10:16.057369
KLM


In [19]:
# Scenario 2: Book a new flight

user_prompt = f"I want to book a flight from {origin} to {destination} on {flight_datetime} with {flight_airline}"
print(ask_and_reply(user_prompt))

# Scenario 3: File a complaint

user_prompt = "This is John Doe. I want to file a complaint about my missed flight. It was an unpleasant surprise. Email me a copy of the complaint to john@doe.com."
print(ask_and_reply(user_prompt))


ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK",\n  "datetime": "2024-03-27 13:10:16.057369",\n  "airline": "KLM"\n}', name='book_flight'), tool_calls=None)
ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "name": "John Doe",\n  "email": "john@doe.com",\n  "text": "I would like to file a complaint about my missed flight. It was an unpleasant surprise."\n}', name='file_complaint'), tool_calls=None)


In [20]:
# --------------------------------------------------------------
# Make It Conversational With Langchain
# --------------------------------------------------------------

llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)

# Start a conversation with multiple requests

user_prompt = """
This is Jane Harris. I am an unhappy customer that wants you to do several things.
First, I neeed to know when's the next flight from Amsterdam to New York.
Please proceed to book that flight for me.
Also, I want to file a complaint about my missed flight. It was an unpleasant surprise. 
Email me a copy of the complaint to jane@harris.com.
Please give me a confirmation after all of these are done.
"""

# Returns the function of the first request (get_flight_info)

/Users/mpaz/Library/Caches/pypoetry/virtualenvs/poetry-030-oHjfFdrQ-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [21]:
# Returns the function of the first request (get_flight_info)

first_response = llm.predict_messages(
    [HumanMessage(content=user_prompt)], functions=function_descriptions_multiple
)

print(first_response)

/Users/mpaz/Library/Caches/pypoetry/virtualenvs/poetry-030-oHjfFdrQ-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content='' additional_kwargs={'function_call': {'arguments': '{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', 'name': 'get_flight_info'}} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 332, 'total_tokens': 358}, 'model_name': 'gpt-3.5-turbo-0613', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}


In [22]:
# Returns the function of the second request (book_flight)
# It takes all the arguments from the prompt but not the returned information

second_response = llm.predict_messages(
    [
        HumanMessage(content=user_prompt),
        AIMessage(content=str(first_response.additional_kwargs)),
        AIMessage(
            role="function",
            additional_kwargs={
                "name": first_response.additional_kwargs["function_call"]["name"]
            },
            content=f"Completed function {first_response.additional_kwargs['function_call']['name']}",
        ),
    ],
    functions=function_descriptions_multiple,
)

print(second_response)


content='' additional_kwargs={'function_call': {'arguments': '{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', 'name': 'get_flight_info'}} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 387, 'total_tokens': 413}, 'model_name': 'gpt-3.5-turbo-0613', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}


In [23]:
# Returns the function of the third request (file_complaint)

third_response = llm.predict_messages(
    [
        HumanMessage(content=user_prompt),
        AIMessage(content=str(first_response.additional_kwargs)),
        AIMessage(content=str(second_response.additional_kwargs)),
        AIMessage(
            role="function",
            additional_kwargs={
                "name": second_response.additional_kwargs["function_call"]["name"]
            },
            content=f"Completed function {second_response.additional_kwargs['function_call']['name']}",
        ),
    ],
    functions=function_descriptions_multiple,
)

print(third_response)


content='' additional_kwargs={'function_call': {'arguments': '{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', 'name': 'get_flight_info'}} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 430, 'total_tokens': 456}, 'model_name': 'gpt-3.5-turbo-0613', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}


In [24]:
# Conversational reply at the end of requests

fourth_response = llm.predict_messages(
    [
        HumanMessage(content=user_prompt),
        AIMessage(content=str(first_response.additional_kwargs)),
        AIMessage(content=str(second_response.additional_kwargs)),
        AIMessage(content=str(third_response.additional_kwargs)),
        AIMessage(
            role="function",
            additional_kwargs={
                "name": third_response.additional_kwargs["function_call"]["name"]
            },
            content=f"Completed function {third_response.additional_kwargs['function_call']['name']}",
        ),
    ],
    functions=function_descriptions_multiple,
)

print(fourth_response)

content='' additional_kwargs={'function_call': {'arguments': '{\n  "loc_origin": "AMS",\n  "loc_destination": "JFK"\n}', 'name': 'get_flight_info'}} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 473, 'total_tokens': 499}, 'model_name': 'gpt-3.5-turbo-0613', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}
